<!-- Main Container (Dark Mode) -->
<div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; color: #e0e0e0; background-color: #1e1e1e; padding: 15px; border-radius: 5px; overflow: hidden;">

<div style="background-color: #2d2d2d; border-radius: 10px; border-left: 5px solid #00d4ff; padding: 20px; box-shadow: 0 4px 15px rgba(0,0,0,0.5);">
        <h1 style="color: #00d4ff; text-align: center; margin: 0; font-weight: 800; text-shadow: 0 0 10px rgba(0, 212, 255, 0.3);">
            🛒 Understanding the Apriori Algorithm
        </h1>
        <p style="text-align: center; font-size: 1.1em; color: #b0bec5; margin-top: 5px;">
            Market Basket Analysis and Association Rule Learning
        </p>
    </div>

<br>

<div style="text-align: justify; line-height: 1.6;">
        <p>
            In the world of data mining, <b style="color: #00d4ff;">Association Rule Learning</b> is a technique used to uncover how items are associated with each other. It is the magic behind "People who bought this also bought that" recommendations.
        </p>
        <p>
            The <b style="color: #00d4ff;">Apriori Algorithm</b> is the classic algorithm used for this purpose. It is designed to operate on databases containing transactions (like a supermarket dataset).
        </p>
    </div>

<hr style="border: 0; border-top: 1px solid #444; margin: 20px 0;">

<h2 style="color: #b388ff; border-bottom: 1px solid #b388ff; padding-bottom: 5px; display: inline-block;">1. The Core Intuition</h2>

<p>Imagine you are a manager at a grocery store. You want to know which products are frequently purchased together.</p>

<div style="background-color: #263238; padding: 15px; border-left: 5px solid #69f0ae; border-radius: 5px; margin-bottom: 20px; color: #eceff1;">
        <b style="color: #69f0ae;">💡 The "Bottom-Up" Approach:</b><br>
        Apriori finds frequent itemsets by:
        <ol style="margin-bottom: 0; padding-left: 20px;">
            <li>Identifying individual items that appear frequently.</li>
            <li>Extending them to larger itemsets as long as they appear frequently enough.</li>
        </ol>
    </div>

 <h2 style="color: #b388ff; border-bottom: 1px solid #b388ff; padding-bottom: 5px; display: inline-block;">2. Key Metrics</h2>

<p>To understand Apriori, you need to grasp three fundamental metrics: <b style="color: #ff8a80;">Support</b>, <b style="color: #ff8a80;">Confidence</b>, and <b style="color: #ff8a80;">Lift</b>.</p>

<table style="width:100%; border-collapse: collapse; margin-bottom: 20px; border: 1px solid #444;">
      <tr style="background-color: #333;">
        <th style="padding: 12px; border: 1px solid #444; text-align: left; color: #fff;">Metric</th>
        <th style="padding: 12px; border: 1px solid #444; text-align: left; color: #fff;">Definition</th>
        <th style="padding: 12px; border: 1px solid #444; text-align: left; color: #fff;">Formula</th>
      </tr>
      <tr>
        <td style="padding: 12px; border: 1px solid #444;"><b style="color: #80d8ff;">📊 Support</b></td>
        <td style="padding: 12px; border: 1px solid #444;">How popular an itemset is.</td>
        <td style="padding: 12px; border: 1px solid #444;">$$ \text{Supp}(A) = \frac{\text{Freq}(A)}{\text{Total}} $$</td>
      </tr>
      <tr>
        <td style="padding: 12px; border: 1px solid #444;"><b style="color: #ea80fc;">🎯 Confidence</b></td>
        <td style="padding: 12px; border: 1px solid #444;">Likelihood of B given A.</td>
        <td style="padding: 12px; border: 1px solid #444;">$$ \text{Conf}(A \rightarrow B) = \frac{\text{Supp}(A \cup B)}{\text{Supp}(A)} $$</td>
      </tr>
      <tr>
        <td style="padding: 12px; border: 1px solid #444;"><b style="color: #ff9e80;">🚀 Lift</b></td>
        <td style="padding: 12px; border: 1px solid #444;">Performance vs Random Chance.</td>
        <td style="padding: 12px; border: 1px solid #444;">$$ \text{Lift} = \frac{\text{Conf}(A \rightarrow B)}{\text{Supp}(B)} $$</td>
      </tr>
    </table>

<h3 style="color: #ff4081; font-size: 1.1em; margin-top: 20px;">Interpreting Lift</h3>
    <ul style="margin-top: 5px; padding-left: 20px;">
        <li><b style="color: #fff;">Lift = 1:</b> No association (independent).</li>
        <li><b style="color: #69f0ae;">Lift > 1:</b> Positive association (likely bought together).</li>
        <li><b style="color: #ff5252;">Lift < 1:</b> Negative association (unlikely bought together).</li>
    </ul>

  <br>

<h2 style="color: #b388ff; border-bottom: 1px solid #b388ff; padding-bottom: 5px; display: inline-block;">3. How the Algorithm Works</h2>

 <div style="background-color: #3e2723; padding: 15px; border-left: 5px solid #ffab40; border-radius: 5px; color: #ffe0b2;">
        <b style="color: #ffab40;">⚠️ The Apriori Principle:</b><br>
        If an itemset is infrequent, then all of its supersets must also be infrequent.<br>
        <i>Example: If {Beer} is infrequent, then {Beer, Diapers} must also be infrequent.</i>
    </div>
    <br>
    <h2 style="color: #b388ff; border-bottom: 1px solid #b388ff; padding-bottom: 5px; display: inline-block;">4. Python Implementation Strategy</h2>

 <p>We typically use the <code>mlxtend</code> library. Here is the standard workflow:</p>

<div style="background-color: #121212; padding: 15px; border-radius: 5px; border: 1px solid #333; font-family: 'Courier New', monospace; color: #a4a4a4; margin-bottom: 25px; box-shadow: inset 0 0 10px #000;">
        <span style="color: #5c6370; font-style: italic;"># --- PSEUDOCODE EXAMPLE ---</span><br>
        <span style="color: #c678dd; font-weight: bold;">from</span> mlxtend.frequent_patterns <span style="color: #c678dd; font-weight: bold;">import</span> apriori, association_rules<br><br>
        <span style="color: #5c6370; font-style: italic;"># 1. Get Frequent Itemsets</span><br>
        frequent_itemsets = apriori(df, min_support=<span style="color: #d19a66;">0.05</span>, use_colnames=<span style="color: #56b6c2;">True</span>)<br><br>
        <span style="color: #5c6370; font-style: italic;"># 2. Generate Rules</span><br>
        rules = association_rules(frequent_itemsets, metric=<span style="color: #98c379;">"lift"</span>, min_threshold=<span style="color: #d19a66;">1</span>)
    </div>

  <h2 style="color: #b388ff; border-bottom: 1px solid #b388ff; padding-bottom: 5px; display: inline-block;">5. Pros and Cons</h2>

 <div style="display: flex; justify-content: space-between; gap: 20px; margin-bottom: 20px; flex-wrap: wrap;">
        <div style="flex: 1; background-color: #1b5e20; padding: 15px; border-radius: 10px; border: 1px solid #2e7d32; min-width: 250px;">
            <h4 style="color: #a5d6a7; text-align: center; margin-top: 0;">✅ Pros</h4>
            <ul style="color: #e8f5e9; padding-left: 20px;">
                <li>Easy to understand and interpret.</li>
                <li>Uses pruning to reduce search space.</li>
            </ul>
        </div>
        <div style="flex: 1; background-color: #b71c1c; padding: 15px; border-radius: 10px; border: 1px solid #c62828; min-width: 250px;">
            <h4 style="color: #ffcdd2; text-align: center; margin-top: 0;">❌ Cons</h4>
            <ul style="color: #ffebee; padding-left: 20px;">
                <li>Computationally expensive on large datasets.</li>
                <li>Requires multiple scans of the database.</li>
            </ul>
        </div>
    </div>

 <div style="clear: both;"></div>

</div>


---


<div style="background-color: #1e1e1e; color: #cfd8dc; padding: 25px; border-radius: 15px; border: 1px solid #37474f; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; box-shadow: 0 10px 20px rgba(0,0,0,0.5);">
    
<div style="text-align: center; border-bottom: 2px solid #7c4dff; padding-bottom: 20px; margin-bottom: 30px;">
        <h1 style="color: #7c4dff; font-weight: 800; margin-bottom: 5px;">
             Dynamic Hashing & Pruning (DHP)
        </h1>
        <p style="font-size: 1.1em; color: #b0bec5; letter-spacing: 1px;">
            Optimizing the <span style="color: #ff4081; font-weight: bold;">$C_2$</span> Bottleneck
        </p>
    </div>

<h2 style="color: #00e5ff; border-left: 4px solid #00e5ff; padding-left: 15px;">1. The Bottleneck: Why optimize $C_2$?</h2>
    <p>
        In standard Apriori, the most expensive step is generating the candidate pairs ($C_2$). Even if you filter out infrequent individual items, the number of possible pairs explodes combinatorially.
    </p>
    <div style="background-color: #263238; padding: 15px; border-radius: 8px; text-align: center; border: 1px dashed #546e7a;">
        <p style="margin: 0; color: #eceff1;">
            If you have 1,000 frequent items, you must check:<br>
            $$ \frac{1000 \times 999}{2} \approx 500,000 \text{ pairs} $$
        </p>
    </div>

 <h2 style="color: #00e5ff; margin-top: 40px; border-left: 4px solid #00e5ff; padding-left: 15px;">2. The Hashing Strategy</h2>
    <p>
        The core idea is to <b>"peek ahead"</b>. While we scan the database to count individual items ($L_1$), we can simultaneously gather rough data about pairs using a <b>Hash Table</b>.
    </p>

<ul style="list-style-type: none; padding: 0;">
        <li style="margin-bottom: 15px; display: flex; align-items: start;">
            <span style="color: #7c4dff; font-weight: bold; margin-right: 10px; font-size: 1.2em;">1.</span>
            <div><b>Scan:</b> As we read a transaction, generate all possible 2-item subsets.</div>
        </li>
        <li style="margin-bottom: 15px; display: flex; align-items: start;">
            <span style="color: #7c4dff; font-weight: bold; margin-right: 10px; font-size: 1.2em;">2.</span>
            <div><b>Hash:</b> Apply a function to map the pair to a specific bucket index.</div>
        </li>
        <li style="margin-bottom: 15px; display: flex; align-items: start;">
            <span style="color: #7c4dff; font-weight: bold; margin-right: 10px; font-size: 1.2em;">3.</span>
            <div><b>Count:</b> Increment the counter for that bucket.</div>
        </li>
    </ul>

<h2 style="color: #00e5ff; margin-top: 40px; border-left: 4px solid #00e5ff; padding-left: 15px;">3. Visual Example</h2>
    
 <p>Assume <b>Min_Support = 3</b> and Hash Function: $h(x,y) = (x \cdot 10 + y) \pmod 7$.</p>

 <table style="width:100%; border-collapse: separate; border-spacing: 0; border: 1px solid #455a64; border-radius: 8px; overflow: hidden;">
        <thead style="background-color: #37474f; color: #00e5ff;">
            <tr>
                <th style="padding: 12px;">Transaction</th>
                <th style="padding: 12px;">Generated Pair</th>
                <th style="padding: 12px;">Hash Calc</th>
                <th style="padding: 12px;">Bucket Index</th>
            </tr>
        </thead>
        <tbody style="color: #b0bec5;">
            <tr style="background-color: #263238;">
                <td rowspan="3" style="padding: 12px; border-bottom: 1px solid #455a64; vertical-align: middle; font-weight: bold;">{1, 2, 3}</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">{1, 2}</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">(12) % 7</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64; color: #ff4081; font-weight: bold;">Bucket 5</td>
            </tr>
            <tr style="background-color: #263238;">
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">{1, 3}</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">(13) % 7</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64; color: #ff4081; font-weight: bold;">Bucket 6</td>
            </tr>
            <tr style="background-color: #263238;">
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">{2, 3}</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64;">(23) % 7</td>
                <td style="padding: 10px; border-bottom: 1px solid #455a64; color: #ff4081; font-weight: bold;">Bucket 2</td>
            </tr>
        </tbody>
    </table>

<div style="margin-top: 30px; padding: 20px; background-color: #212121; border-left: 5px solid #ff4081; border-radius: 0 10px 10px 0;">
        <h3 style="margin-top: 0; color: #ff4081;">✂️ The Pruning Rule</h3>
        <p>
            When creating $C_2$, we check the bucket count for every candidate pair.
        </p>
        <p style="font-size: 1.1em; font-weight: bold; color: #fff;">
            IF <code>Bucket_Count < Min_Support</code> THEN Discard Pair.
        </p>
        <p style="font-size: 0.9em; color: #90caf9;">
            <i>Why? Because even if items A and B are individually frequent, if their combined bucket count is low, they mathematically <b>cannot</b> be frequent together.</i>
        </p>
    </div>

<h2 style="color: #00e5ff; margin-top: 40px; border-left: 4px solid #00e5ff; padding-left: 15px;">4. The "Collision" Catch</h2>
    <p>
        Sometimes, two infrequent pairs land in the same bucket, making the bucket count artificially high (False Positive).
    </p>
    
<div style="display: flex; gap: 15px; margin-top: 15px;">
        <div style="flex: 1; background-color: #1b5e20; padding: 15px; border-radius: 8px; color: #a5d6a7;">
            <b>Low Bucket Count:</b><br>
            100% Safe to Prune. <br>The pair is definitely infrequent.
        </div>
        <div style="flex: 1; background-color: #b71c1c; padding: 15px; border-radius: 8px; color: #ffcdd2;">
            <b>High Bucket Count:</b><br>
            Might be frequent, or might be a collision.<br> We must keep it and check later.
        </div>
    </div>

</div>


<!-- Main Container (Dark Mode) -->
<div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; color: #e0e0e0; background-color: #1e1e1e; padding: 20px; border-radius: 10px; overflow: hidden; margin-bottom: 30px; clear: both;">

<div style="background-color: #2d2d2d; border-radius: 10px; border-left: 5px solid #00e676; padding: 20px; box-shadow: 0 4px 15px rgba(0,0,0,0.5); margin-bottom: 25px;">
        <h1 style="color: #00e676; text-align: center; margin: 0; font-weight: 800; text-shadow: 0 0 10px rgba(0, 230, 118, 0.3);">
            🎲 Sampling in Apriori
        </h1>
        <p style="text-align: center; font-size: 1.1em; color: #b0bec5; margin-top: 5px;">
            Reducing I/O Overhead by Limiting the Search Domain
        </p>
    </div>

<h2 style="color: #b388ff; border-bottom: 1px solid #b388ff; padding-bottom: 5px; display: inline-block;">1. The Core Concept</h2>
    <p style="line-height: 1.6;">
        The standard Apriori algorithm suffers from high <b>I/O costs</b> because it scans the entire database ($D$) multiple times. If $D$ is massive (e.g., 10 million transactions), reading from the disk becomes the bottleneck.
    </p>
    <p>
        <b>Sampling</b> solves this by picking a random subset of transactions ($S$) that fits entirely into <b>Main Memory (RAM)</b>.
    </p>

 <div style="background-color: #263238; padding: 15px; border-radius: 8px; border-left: 4px solid #4fc3f7; margin: 20px 0;">
        <b style="color: #4fc3f7;">🥣 The Analogy:</b><br>
        To check if a giant pot of soup is salty, you don't drink the whole pot (scan the whole DB). You take a <b>spoonful (sample)</b>. If the spoonful is salty, the pot is likely salty.
    </div>

<h2 style="color: #b388ff; border-bottom: 1px solid #b388ff; padding-bottom: 5px; display: inline-block; margin-top: 20px;">2. The Risk: False Negatives</h2>
    
<p>
        The danger of sampling is missing a frequent itemset just because it didn't appear enough times in your specific random sample. This is called a <b style="color: #ff5252;">False Negative</b>.
    </p>

<div style="background-color: #3e2723; padding: 15px; border-radius: 8px; border: 1px solid #ffab40; margin-bottom: 25px;">
        <h3 style="color: #ffab40; margin-top: 0;">🛠️ The Solution: Lowered Threshold</h3>
        <p style="margin-bottom: 10px;">
            To mitigate this risk, we use a lower support threshold for the sample ($s'$) than we do for the actual database ($s$).
        </p>
        <div style="background-color: #000; padding: 10px; border-radius: 4px; text-align: center; font-size: 1.2em; color: #fff;">
            $$ s' < s $$
        </div>
        <p style="margin-top: 10px; font-size: 0.9em; color: #ffe0b2;">
            <i>Example: If the global requirement is 5% ($s$), we might search the sample using 3% ($s'$). This casts a "wider net" to catch borderline candidates.</i>
        </p>
    </div>

<h2 style="color: #b388ff; border-bottom: 1px solid #b388ff; padding-bottom: 5px; display: inline-block;">3. The Workflow (Two-Phase Approach)</h2>

<div style="display: flex; flex-direction: column; gap: 15px; margin-top: 15px;">
        
 <div style="display: flex; align-items: center; background-color: #1c2a36; padding: 15px; border-radius: 8px; border-left: 5px solid #00e5ff;">
            <div style="font-size: 2em; margin-right: 20px;">🕵️</div>
            <div>
                <b style="color: #00e5ff; font-size: 1.1em;">Phase 1: Guessing (in RAM)</b>
                <p style="margin: 5px 0; font-size: 0.95em; color: #cfd8dc;">
                    Extract sample $S$. Run Apriori on $S$ using the lowered threshold ($s'$). The output is a list of <b>Candidate Itemsets</b>.
                </p>
            </div>
        </div>

<div style="display: flex; align-items: center; background-color: #1c2a36; padding: 15px; border-radius: 8px; border-left: 5px solid #7c4dff;">
            <div style="font-size: 2em; margin-right: 20px;">✅</div>
            <div>
                <b style="color: #7c4dff; font-size: 1.1em;">Phase 2: Verification (on Disk)</b>
                <p style="margin: 5px 0; font-size: 0.95em; color: #cfd8dc;">
                    Scan the full Database $D$ <b>exactly once</b>. Count the frequency of the Candidates found in Phase 1. Keep only those that meet the original threshold ($s$).
                </p>
            </div>
        </div>
 </div>

 <br>
    <div style="text-align: center; font-size: 0.9em; color: #757575;">
        <i>Note: While we might still get False Negatives (rarely), we completely eliminate False Positives in Phase 2.</i>
    </div>

</div>


<div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; color: #e0e0e0; background-color: #1e1e1e; padding: 15px; border-radius: 10px; overflow: hidden; margin-bottom: 20px;">

<h2 style="color: #00e5ff; border-bottom: 2px solid #00e5ff; padding-bottom: 10px; margin-top: 0;">
         Step 1: Data Structure Strategy
    </h2>

 <p>
        To implement Apriori efficiently from scratch, we cannot use standard Python lists for the transactions. Instead, we convert our dataset into a <b>List of Frozensets</b>.
    </p>

<div style="background-color: #263238; padding: 15px; border-left: 4px solid #ff4081; border-radius: 5px; margin: 20px 0;">
        <strong style="color: #ff4081; font-size: 1.1em;">Why `frozenset`?</strong>
        <ul style="margin-top: 10px; margin-bottom: 0; padding-left: 20px; line-height: 1.6;">
            <li>
                <b>Immutable Keys:</b> Our final output requires a dictionary: <code>{itemset: count}</code>. In Python, dictionary keys <i>must</i> be immutable (unchangeable). Standard lists and sets are mutable, so they throw errors. <code>frozenset</code> works perfectly.
            </li>
            <li>
                <b>O(1) Speed:</b> Checking if a transaction contains a candidate (subset check) is significantly faster with set math than iterating through lists.
            </li>
        </ul>
    </div>

<div style="background-color: #121212; color: #a6accd; padding: 10px; border-radius: 5px; font-family: 'Courier New', monospace; border: 1px solid #333;">
        <span style="color: #757575;"># Raw Data (List of Lists)</span><br>
        raw = [<span style="color: #c3e88d;">'Milk'</span>, <span style="color: #c3e88d;">'Bread'</span>]<br><br>
        <span style="color: #757575;"># Optimized Data (Frozenset)</span><br>
        optimized = <span style="color: #82aaff;">frozens</span>([<span style="color: #c3e88d;">'Milk'</span>, <span style="color: #c3e88d;">'Bread'</span>])
    </div>

</div>



In [1]:
def load_dataset():
    """
    Creates a dummy dataset for testing.
    Returns a list of transactions, where each transaction is a frozenset.
    """
    # Raw data: List of lists
    raw_data = [
        ['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
        ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
        ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
        ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
        ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']
    ]
    
    # Converting to List of frozensets
    # We use frozenset to make sure items are unique in a transaction 
    # and immutable (hashable) for dictionary keys later.
    dataset = [frozenset(transaction) for transaction in raw_data]
    
    return dataset

# Test the step
dataset = load_dataset()
print(f"Number of transactions: {len(dataset)}")
print(f"First transaction type: {type(dataset[0])}")
print(f"First transaction content: {dataset[0]}")
# later on this project, we can use other data sets and freez them for using it inside of work.

Number of transactions: 5
First transaction type: <class 'frozenset'>
First transaction content: frozenset({'Yogurt', 'Onion', 'Kidney Beans', 'Nutmeg', 'Milk', 'Eggs'})


<!-- Main Container with isolation to prevent layout conflicts -->
<div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; color: #e0e0e0; background-color: #1e1e1e; padding: 20px; border-radius: 10px; overflow: hidden; margin-bottom: 20px; clear: both;">

<h2 style="color: #b388ff; border-bottom: 2px solid #b388ff; padding-bottom: 10px; margin-top: 0;">
        🗄️ Alternative Data Structures
    </h2>

 <p style="font-size: 0.95em; line-height: 1.6;">
        While <code>frozenset</code> is the Pythonic standard for Apriori, other structures exist. The choice depends on <b>Memory</b> vs. <b>Speed</b> trade-offs.
    </p>

 <div style="margin-top: 20px; border: 1px solid #333; border-radius: 8px; background-color: #252526;">
        <div style="padding: 10px; background-color: #303031; border-bottom: 1px solid #333; font-weight: bold; color: #00e5ff;">
            1. Binary Matrix (One-Hot Encoding)
        </div>
        <div style="padding: 15px;">
            <p style="margin-top: 0; font-size: 0.9em;">
                Instead of lists, we create a matrix of 0s and 1s (Rows=Transactions, Cols=Items).
            </p>
            <!-- Code Visual -->
            <div style="background-color: #121212; padding: 10px; border-radius: 4px; font-family: monospace; font-size: 0.85em; color: #a6accd; margin-bottom: 10px;">
#    Milk  Bread  Beer <br>
T1 [  1,     1,     0  ]<br>
T2 [  0,     1,     1  ]
            </div>
            <ul style="font-size: 0.9em; margin-bottom: 0; padding-left: 20px;">
                <li style="color: #69f0ae; margin-bottom: 5px;"><b>✅ Pro:</b> Extremely fast calculation using Vector/Matrix math (NumPy).</li>
                <li style="color: #ff5252;"><b>❌ Con:</b> <b>Memory Hog.</b> If you have 10k items but a user buys only 5, you store 9,995 useless zeros.</li>
            </ul>
        </div>
    </div>

 <div style="margin-top: 20px; border: 1px solid #333; border-radius: 8px; background-color: #252526;">
        <div style="padding: 10px; background-color: #303031; border-bottom: 1px solid #333; font-weight: bold; color: #ea80fc;">
            2. Vertical Layout (TID-List)
        </div>
        <div style="padding: 15px;">
            <p style="margin-top: 0; font-size: 0.9em;">
                Instead of storing "What is in T1?", we store "Where is Item A?" (Inverted Index).
            </p>
            <div style="background-color: #121212; padding: 10px; border-radius: 4px; font-family: monospace; font-size: 0.85em; color: #a6accd; margin-bottom: 10px;">
Milk  -> {T1, T4, T5}<br>
Bread -> {T1, T2}
            </div>
            <ul style="font-size: 0.9em; margin-bottom: 0; padding-left: 20px;">
                <li style="color: #69f0ae; margin-bottom: 5px;"><b>✅ Pro:</b> No need to scan database rows. Just find the <b>Intersection</b> of sets.</li>
                <li style="color: #ffb74d;"><b>⚠️ Note:</b> This is used in the <b>Eclat</b> algorithm, not standard Apriori.</li>
            </ul>
        </div>
    </div>

 <div style="margin-top: 20px; padding-top: 10px; border-top: 1px dashed #555; font-style: italic; font-size: 0.9em; color: #9e9e9e;">
        <b>Verdict:</b> We stick to <code>frozenset</code> because it handles <b>Sparse Data</b> (real-world scenario) efficiently without the complexity of Eclat or the memory cost of Matrices.
    </div>

</div>


<div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; color: #e0e0e0; background-color: #1e1e1e; padding: 20px; border-radius: 10px; overflow: hidden; margin-bottom: 20px; clear: both;">

<h2 style="color: #ffab40; border-bottom: 2px solid #ffab40; padding-bottom: 10px; margin-top: 0;">
        🎛️ Parameter: `min_sup_fraction`
    </h2>

<p style="font-size: 0.95em; line-height: 1.6;">
        This parameter is the <b>"Popularity Threshold"</b>. It dictates how frequent an item must be to be considered "interesting."
    </p>

<div style="display: flex; gap: 15px; margin-top: 20px; flex-wrap: wrap;">
        
<div style="flex: 1; min-width: 250px; background-color: #263238; padding: 15px; border-radius: 8px; border-left: 4px solid #00e676;">
            <strong style="color: #00e676; display: block; margin-bottom: 8px;">The Logic</strong>
            If you set <code>0.4</code> (40%), you are telling the algorithm:
            <br><br>
            <i>"Only keep items that appear in at least <b>40%</b> of all transactions. Discard the rest."</i>
        </div>

<div style="flex: 1; min-width: 250px; background-color: #263238; padding: 15px; border-radius: 8px; border-left: 4px solid #2979ff;">
            <strong style="color: #2979ff; display: block; margin-bottom: 8px;">Why a Fraction?</strong>
            Using a fraction makes your code <b>Scalable</b>.
            <ul style="margin-bottom: 0; padding-left: 20px; font-size: 0.9em; margin-top: 5px;">
                <li><b>Fixed Count (e.g., 5):</b> Good for small data, bad if data grows to 1 million.</li>
                <li><b>Fraction (e.g., 0.1):</b> Automatically adjusts the threshold as the database grows.</li>
            </ul>
        </div>
    </div>

<h3 style="color: #e0e0e0; margin-top: 25px; font-size: 1.1em;">⚙️ Implementation Strategy</h3>
    <p style="font-size: 0.9em;">
        Computers prefer integers for counting. Before starting the loop, we convert the fraction into a solid number called <code>min_support_count</code>.
    </p>

<div style="background-color: #121212; border: 1px dashed #555; padding: 15px; text-align: center; font-family: 'Courier New', monospace; font-size: 1.1em; border-radius: 5px; color: #ffab40;">
        min_count = Total_Transactions × min_sup_fraction
    </div>

 <div style="margin-top: 15px; font-size: 0.9em; color: #bdbdbd;">
        <b>Example:</b>
        If we have <span style="color: #fff;">5 transactions</span> and <span style="color: #fff;">min_sup = 0.6</span>:
        <br>
        Calculation: <code>5 * 0.6 = 3.0</code>
        <br>
        <span style="color: #69f0ae;">Result:</span> An item must appear <b>3 times</b> to survive.
    </div>

</div>


In [2]:
def get_min_support_count(dataset, min_sup_fraction):
    """
    Converts the fractional support (e.g., 0.6) into an integer count (e.g., 3).
    """
    total_transactions = len(dataset)
    
    # Calculate count
    min_support_count = total_transactions * min_sup_fraction
    
    # Note: We can use math.ceil if we want to be strict, 
    # but usually simple multiplication is enough for basic implementation.
    # Let's print it to see what's happening.
    print(f"Total Transactions: {total_transactions}")
    print(f"Min Support Fraction: {min_sup_fraction}")
    print(f"Calculated Min Support Count: {min_support_count}")
    
    return min_support_count

# Test with our dummy dataset
# dataset is from Step 1
min_count = get_min_support_count(dataset, 0.6)


Total Transactions: 5
Min Support Fraction: 0.6
Calculated Min Support Count: 3.0


<div style="
    background-color: #1e1e1e; 
    color: #e0e0e0; 
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; 
    padding: 25px; 
    border-radius: 12px; 
    border: 1px solid #333; 
    overflow: hidden; 
    box-shadow: 0 4px 15px rgba(0,0,0,0.5);
    line-height: 1.6;
    margin-bottom: 20px;">

<div style="border-bottom: 2px solid #00d4ff; padding-bottom: 15px; margin-bottom: 25px;">
        <h2 style="color: #00d4ff; margin: 0; font-size: 28px; letter-spacing: 1px;">
            Understanding the Core Cycle: <span style="color: #ffab40;">C</span> vs <span style="color: #00e676;">L</span>
        </h2>
        <p style="color: #b0b0b0; margin-top: 8px; font-size: 15px;">
            The Apriori algorithm isn't a linear path; it's a cycle of generation and survival.
        </p>
    </div>

<div style="display: flex; gap: 20px; flex-wrap: wrap; margin-bottom: 30px;">
        
<div style="
            flex: 1; 
            min-width: 280px; 
            background-color: #2d2d2d; 
            padding: 20px; 
            border-radius: 8px; 
            border-left: 5px solid #ffab40;
            box-shadow: 0 2px 5px rgba(0,0,0,0.3);">
            <h3 style="color: #ffab40; margin-top: 0; display: flex; align-items: center;">
                <span style="font-size: 24px; margin-right: 10px;">🧪</span>
                $C_k$ (Candidate Sets)
            </h3>
            <p style="font-size: 14px; color: #ccc;">
                <b>The "Maybe" Pile.</b><br>
                These are the itemsets that <i>might</i> be frequent. We construct them blindly based on the winners of the previous round.
            </p>
            <ul style="font-size: 13px; color: #aaa; padding-left: 20px;">
                <li>Contains both trash and treasure.</li>
                <li>Must be counted against the full database.</li>
                <li><b>Size:</b> Usually much larger than $L_k$.</li>
            </ul>
        </div>

<div style="
            flex: 1; 
            min-width: 280px; 
            background-color: #2d2d2d; 
            padding: 20px; 
            border-radius: 8px; 
            border-left: 5px solid #00e676;
            box-shadow: 0 2px 5px rgba(0,0,0,0.3);">
            <h3 style="color: #00e676; margin-top: 0; display: flex; align-items: center;">
                <span style="font-size: 24px; margin-right: 10px;">🏆</span>
                $L_k$ (Frequent Sets)
            </h3>
            <p style="font-size: 14px; color: #ccc;">
                <b>The "Survivors".</b><br>
                These are the candidates that survived the support threshold check. They are proven to be popular.
            </p>
            <ul style="font-size: 13px; color: #aaa; padding-left: 20px;">
                <li>Only contains itemsets with count $\ge$ min_support.</li>
                <li>Used as "parents" to generate the next generation ($k+1$).</li>
                <li><b>Size:</b> The final output we care about.</li>
            </ul>
        </div>
    </div>

<div style="background-color: #181818; padding: 20px; border-radius: 10px; text-align: center; border: 1px dashed #444;">
        <h4 style="color: #fff; margin-top: 0;">The Iterative Process (Step $k$)</h4>
        
<div style="display: flex; justify-content: center; align-items: center; flex-wrap: wrap; gap: 15px; font-weight: bold;">
            
<div style="padding: 10px 15px; background: #2a2a2a; border: 1px solid #00e676; color: #00e676; border-radius: 6px;">
                $L_{k-1}$<br><span style="font-size: 10px; color: #888;">Previous Winners</span>
            </div>

<div style="font-size: 20px; color: #555;">➜</div>

<div style="padding: 10px 15px; background: #2a2a2a; border: 1px solid #00d4ff; color: #00d4ff; border-radius: 6px;">
                JOIN<br><span style="font-size: 10px; color: #888;">Create Pairs/Triples</span>
            </div>

 <div style="font-size: 20px; color: #555;">➜</div>

<div style="padding: 10px 15px; background: #2a2a2a; border: 1px solid #ffab40; color: #ffab40; border-radius: 6px;">
                $C_k$<br><span style="font-size: 10px; color: #888;">New Candidates</span>
            </div>

<div style="font-size: 20px; color: #555;">➜</div>

<div style="padding: 10px 15px; background: #2a2a2a; border: 1px solid #ff5252; color: #ff5252; border-radius: 6px;">
                FILTER<br><span style="font-size: 10px; color: #888;">Scan DB & Prune</span>
            </div>

 <div style="font-size: 20px; color: #555;">➜</div>

<div style="padding: 10px 15px; background: #2a2a2a; border: 1px solid #00e676; color: #00e676; border-radius: 6px; box-shadow: 0 0 10px rgba(0, 230, 118, 0.2);">
                $L_k$<br><span style="font-size: 10px; color: #888;">Current Winners</span>
            </div>
</div>
    </div>
    
<div style="margin-top: 15px; font-size: 13px; color: #777; text-align: center;">
        The subscript <b>$k$</b> simply represents the size of the itemset (e.g., $k$ means we are looking at pairs).
    </div>

</div>



In [3]:
def create_c1(dataset):
    
    c1_counts = {}
    
    for transaction in dataset:
        for item in transaction:
            # Important: Wrap the single item in a frozenset to make it an itemset
            itemset = frozenset([item]) 
            
            if itemset in c1_counts:
                c1_counts[itemset] += 1
            else:
                c1_counts[itemset] = 1
                
    return c1_counts


C1 = create_c1(dataset)

print(f"Number of unique items in C1: {len(C1)}")
print("\nFirst 5 candidates in C1:")
for item, count in list(C1.items())[:5]:
    print(f"{list(item)}: {count}")


Number of unique items in C1: 11

First 5 candidates in C1:
['Yogurt']: 3
['Onion']: 3
['Kidney Beans']: 5
['Nutmeg']: 2
['Milk']: 3


- In the Apriori algorithm, we always work with Sets. Even when the item is a singleton, we should not store it as the string 'Milk', but rather as the set {'Milk'} (or frozenset(['Milk'])).This keeps our data structure consistent in the later steps (when we create pairs and triplets).

In [4]:
def filter_candidates(candidate_counts, min_support_count):
    
    l_counts = {}
    
    for itemset, count in candidate_counts.items():
        if count >= min_support_count:
            l_counts[itemset] = count
            
    return l_counts

# --- Test Step 2: Create L1 from C1 ---

# assume support = 2 
MIN_SUPPORT_COUNT = 2 

#
L1 = filter_candidates(C1, MIN_SUPPORT_COUNT)

print(f"Original Candidates (C1): {len(C1)}")
print(f"Filtered Frequent Itemsets (L1): {len(L1)}")
print("\nItems in L1:")
for item, count in L1.items():
    print(f"{list(item)}: {count}")


Original Candidates (C1): 11
Filtered Frequent Itemsets (L1): 7

Items in L1:
['Yogurt']: 3
['Onion']: 3
['Kidney Beans']: 5
['Nutmeg']: 2
['Milk']: 3
['Eggs']: 4
['Corn']: 2


In [5]:
def generate_Ck(l_prev, k):
    """
    Generates new candidates (Ck) by joining the previous frequent itemsets (L_k-1).
    
    Args:
        l_prev: The dictionary of frequent itemsets from the previous step (L1, L2...).
        k: The size of the itemsets we want to generate (e.g., 2 for pairs).
        
    Returns:
        candidates: A set of frozensets (the new candidates).
    """
    candidates = set()
    # Extract just the itemsets (keys) from the dictionary
    l_itemsets = list(l_prev.keys())
    
    n = len(l_itemsets)
    
    # Double loop to compare every itemset with every other itemset
    for i in range(n):
        for j in range(i + 1, n):
            
            # Join step: Combine two itemsets
            itemset1 = list(l_itemsets[i])
            itemset2 = list(l_itemsets[j])
            
            # Standard Apriori Join Logic:
            # For k=2, we just join every pair.
            # For k>2, we typically join if the first k-2 items are the same.
            # Here, we use a simple Set Union approach which works generally.
            union_set = l_itemsets[i] | l_itemsets[j]
            
            # Check if the new set has the correct length 'k'
            if len(union_set) == k:
                # Optimization: We could check if all subsets are in l_prev (Pruning),
                # but for simplicity in this step, we just add it.
                candidates.add(union_set)
                
    return candidates

# --- Test Step 3: Generate C2 from L1 ---

# We want pairs, so k=2
C2_candidates = generate_Ck(L1, k=2)

print(f"Number of candidates in C2: {len(C2_candidates)}")
print("\nFirst 5 candidates in C2 (Empty buckets, not counted yet):")
for cand in list(C2_candidates)[:5]:
    print(list(cand))


Number of candidates in C2: 21

First 5 candidates in C2 (Empty buckets, not counted yet):
['Kidney Beans', 'Eggs']
['Milk', 'Eggs']
['Kidney Beans', 'Nutmeg']
['Milk', 'Nutmeg']
['Yogurt', 'Eggs']


In [6]:
def count_candidates(dataset, candidates):
    """
    Scans the database and counts the frequency of the given candidates.
    
    Args:
        dataset: The list of transactions (frozensets).
        candidates: A list/set of itemsets (frozensets) we want to count.
        
    Returns:
        candidate_counts: A dictionary { itemset: count }
    """
    candidate_counts = {}
    
    # for better parsing I used list
    cand_list = list(candidates)
    
    for transaction in dataset:
        for cand in cand_list:
            if cand.issubset(transaction):
                if cand in candidate_counts:
                    candidate_counts[cand] += 1
                else:
                    candidate_counts[cand] = 1
                    
    return candidate_counts

# --- Test Step 4: Count C2 ---

C2_counts = count_candidates(dataset, C2_candidates)

print(f"Candidates counted: {len(C2_counts)}")
print("\nTop 5 most frequent pairs in C2:")
sorted_c2 = sorted(C2_counts.items(), key=lambda x: x[1], reverse=True)

for item, count in sorted_c2[:5]:
    print(f"{list(item)}: {count}")


Candidates counted: 20

Top 5 most frequent pairs in C2:
['Kidney Beans', 'Eggs']: 4
['Onion', 'Kidney Beans']: 3
['Kidney Beans', 'Yogurt']: 3
['Milk', 'Kidney Beans']: 3
['Onion', 'Eggs']: 3


In [7]:
# --- Test Step 5: Create L2 from C2_counts ---

# MIN_SUPPORT_COUNT = 2
L2 = filter_candidates(C2_counts, MIN_SUPPORT_COUNT)

print(f"Candidates analyzed (C2): {len(C2_counts)}")
print(f"Frequent Pairs kept (L2): {len(L2)}")
print("\nItems in L2 (The Winners of Round 2):")
sorted_l2 = sorted(L2.items(), key=lambda x: x[1], reverse=True)

for item, count in sorted_l2:
    print(f"{list(item)}: {count}")


Candidates analyzed (C2): 20
Frequent Pairs kept (L2): 14

Items in L2 (The Winners of Round 2):
['Kidney Beans', 'Eggs']: 4
['Onion', 'Kidney Beans']: 3
['Kidney Beans', 'Yogurt']: 3
['Milk', 'Kidney Beans']: 3
['Onion', 'Eggs']: 3
['Milk', 'Eggs']: 2
['Kidney Beans', 'Nutmeg']: 2
['Yogurt', 'Eggs']: 2
['Onion', 'Nutmeg']: 2
['Milk', 'Yogurt']: 2
['Nutmeg', 'Eggs']: 2
['Yogurt', 'Nutmeg']: 2
['Onion', 'Yogurt']: 2
['Kidney Beans', 'Corn']: 2


- until now, we have only operated on the 2nd itemset!

In [8]:
import time
import sys
import tracemalloc  # اضافه شدن کتابخانه مدیریت حافظه

def apriori(dataset, min_support_fraction=0.1):
    
    # --- START MEMORY MONITORING ---
    tracemalloc.start()
    
    start_time = time.time()
    
    # 1. Calculate absolute support count
    min_support_count = int(len(dataset) * min_support_fraction)
    print(f"DEBUG: Min Support Count = {min_support_count} (Total Rows: {len(dataset)})")

    L_global = {} # To store results from all k steps (L1, L2, L3...)
    
    # 2. Step k=1: Create C1 and L1
    print("--- Processing k=1 ---")
    C1 = create_c1(dataset)
    L1 = filter_candidates(C1, min_support_count)
    
    # Add L1 to global results
    L_global.update(L1)
    
    # Current Frequent Itemset List (L_current starts as L1)
    L_current = L1
    k = 2
    
    # Main Loop (for k=2, 3, ...)
    while len(L_current) > 0:
        print(f"--- Processing k={k} ---")
        
        # Generate Candidates (Ck) from previous frequent itemsets (L_k-1)
        Ck_candidates = generate_Ck(L_current, k)
        
        if not Ck_candidates:
            print(f"DEBUG: No candidates generated for k={k}. Stopping.")
            break
            
        # Count Candidates (Scan Database)
        Ck_counts = count_candidates(dataset, Ck_candidates)
        
        # Filter Candidates to get Lk
        Lk = filter_candidates(Ck_counts, min_support_count)
        
        if not Lk:
            print(f"DEBUG: No frequent itemsets found for k={k}. Stopping.")
            break
            
        # Save results and update for next iteration
        L_global.update(Lk)
        L_current = Lk # Lk becomes the parent for next round
        
        print(f"Found {len(Lk)} frequent itemsets of size {k}.")
        k += 1

    end_time = time.time()
    
    # --- STOP MEMORY MONITORING & GET PEAK ---
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    # Stats
    stats = {
        "execution_time": end_time - start_time,
        "memory_peak_mb": peak / (1024 * 1024) # Convert Bytes to MB
    }
    
    return L_global, stats

# --- FINAL TEST ---

# Let's run the full algorithm!
# Using 0.18 (approx 2 transactions) to match our previous manual steps
all_frequent_itemsets, metrics = apriori(dataset, min_support_fraction=0.18)

print("\n================ RESULT ================")
print(f"Total Frequent Itemsets Found: {len(all_frequent_itemsets)}")
print(f"Execution Time: {metrics['execution_time']:.4f} seconds")
print(f"Peak Memory:    {metrics['memory_peak_mb']:.6f} MB") # \
print("========================================")

# Print top 10 itemsets found
sorted_results = sorted(all_frequent_itemsets.items(), key=lambda x: x[1], reverse=True)
print("\nTop 10 Frequent Itemsets:")
for item, count in sorted_results[:10]:
    print(f"{list(item)}: {count}")


DEBUG: Min Support Count = 0 (Total Rows: 5)
--- Processing k=1 ---
--- Processing k=2 ---
Found 36 frequent itemsets of size 2.
--- Processing k=3 ---
Found 51 frequent itemsets of size 3.
--- Processing k=4 ---
Found 36 frequent itemsets of size 4.
--- Processing k=5 ---
Found 13 frequent itemsets of size 5.
--- Processing k=6 ---
Found 2 frequent itemsets of size 6.
--- Processing k=7 ---
DEBUG: No frequent itemsets found for k=7. Stopping.

================ RESULT ================
Total Frequent Itemsets Found: 149
Execution Time: 0.0254 seconds
Peak Memory:    0.120693 MB

Top 10 Frequent Itemsets:
['Kidney Beans']: 5
['Eggs']: 4
['Kidney Beans', 'Eggs']: 4
['Yogurt']: 3
['Onion']: 3
['Milk']: 3
['Onion', 'Kidney Beans']: 3
['Kidney Beans', 'Yogurt']: 3
['Milk', 'Kidney Beans']: 3
['Onion', 'Eggs']: 3
